# Package loading

In [105]:
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation, add, concatenate

In [2]:
tf.version

<module 'tensorflow._api.v2.version' from 'C:\\OSGEO4~1\\apps\\Python37\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

# Datasets Manipulations

In [51]:
import os
from glob import glob
import numpy as np
from osgeo import gdal
from matplotlib import pyplot as plt

In [60]:
path = u'../../'
width = 256
num = 250

In [53]:
def load_data(path):
    images_path = glob(os.path.join(path, "tiles/*"))
    masks_path = glob(os.path.join(path, "masks/*"))
    return images_path, masks_path

In [83]:
def get_raster(raster_path):
    ds = gdal.Open(raster_path)
    data = np.empty((ds.RasterYSize, ds.RasterXSize, ds.RasterCount), dtype=np.float32)
    for b in range(1, ds.RasterCount + 1):
        band = ds.GetRasterBand(b).ReadAsArray()
        data[:, :, b-1] = band
    if data.shape[-1] > 1:
        data = norma_data(data, norma_methods='min-max')   
    return data

In [84]:
def norma_data(data, norma_methods="z-score"):
    arr = np.empty(data.shape, dtype=np.float32)
    for i in range(data.shape[-1]):
        array = data.transpose(2, 0, 1)[i, :, :]
        mins, maxs, mean, std= np.percentile(array, 1), np.percentile(array, 99), np.mean(array), np.std(array)
        if norma_methods == "z-score":
            new_array = (array-mean)/std
        else:
            new_array = np.clip(2*(array-mins)/(maxs-mins), 0, 1)
        arr[:, :, i] = new_array
    return arr

In [89]:
def pare_fun(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()
    
        x = get_raster(x)
        y = get_raster(y)
        return x, y
    
    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    image.set_shape([1000, 1000, 7])
    mask.set_shape([1000, 1000, 1])
    return image, mask

In [95]:
def dataset_generator(path, width, num):
    images_path, masks_path = load_data(path)
    datasets = tf.data.Dataset.from_tensor_slices((images_path, masks_path))
    ds = datasets.map(pare_fun)
    for images, masks in ds:
        for i in range(num):
            location = np.random.randint(width/2 - 1, 999 - width/2, (2,))
            h, w = location[0], location[1]
            d1 = int(width/2 - 1)
            d2 = int(width - d1)
            image_patch = images[h-d1: h+d2, w-d1: w+d2, :]
            mask_patch = masks[h-d1: h+d2, w-d1: w+d2]
            yield image_patch, mask_patch

In [96]:
dataset_generator = dataset_generator(path, width, num)

In [93]:
for image_patch, mask_patch in dataset_generator:
    print(images_patch.shape, masks_patch.shape)

In [ ]:
# plt.imshow(first_sample['mask'][:, :])
# plt.show()

In [ ]:
# plt.imshow(first_sample['image'][:, :, 0:3])
# plt.show()

# Model Construction

In [99]:
def res_block(x, nb_filters, strides):
    res_path = BatchNormalization()(x)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same', strides=strides[0])(res_path)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same', strides=strides[1])(res_path)

    shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1), strides=strides[0])(x)
    shortcut = BatchNormalization()(shortcut)

    res_path = add([shortcut, res_path])
    return res_path

In [100]:
def encoder(x):
    to_decoder = []

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1))(x)
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1))(main_path)

    shortcut = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1))(x)
    shortcut = BatchNormalization()(shortcut)

    main_path = add([shortcut, main_path])
    # first branching to decoder
    to_decoder.append(main_path)

    main_path = res_block(main_path, [128, 128], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    main_path = res_block(main_path, [256, 256], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    return to_decoder

In [101]:
def decoder(x, from_encoder):
    main_path = UpSampling2D(size=(2, 2))(x)
    main_path = concatenate([main_path, from_encoder[2]], axis=3)
    main_path = res_block(main_path, [256, 256], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = concatenate([main_path, from_encoder[1]], axis=3)
    main_path = res_block(main_path, [128, 128], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = concatenate([main_path, from_encoder[0]], axis=3)
    main_path = res_block(main_path, [64, 64], [(1, 1), (1, 1)])

    return main_path

In [102]:
def build_res_unet(input_shape):
    inputs = Input(shape=input_shape)

    to_decoder = encoder(inputs)

    path = res_block(to_decoder[2], [512, 512], [(2, 2), (1, 1)])

    path = decoder(path, from_encoder=to_decoder)

    path = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(path)

    return Model(inputs=inputs, outputs=path)

In [103]:
input_shape = (256, 256, 7)

In [106]:
model = build_res_unet(input_shape=input_shape)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 7) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 4096        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 512         input_1[0][0]                    
______________________________________________________________________________________________

# Model Compile

In [ ]:
# Optimizer, Loss and metrics
# Model.compile()

# Model Training

In [ ]:
# Model.fit()